<a href="https://colab.research.google.com/github/Matinnorouzi2023/Leetcode-Python/blob/main/TradingEnviroment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np  # وارد کردن کتابخانه numpy برای عملیات‌های محاسباتی
import random  # وارد کردن کتابخانه random برای تولید اعداد تصادفی

# تعریف کلاس محیط معامله‌گری
class TradingEnvironment:
    # متد سازنده کلاس که با قیمت اولیه و بازه قیمت فراخوانی می‌شود
    def __init__(self, initial_price=100, price_range=(90, 110), max_steps=100):
        self.initial_price = initial_price  # ذخیره قیمت اولیه
        self.current_price = initial_price  # تنظیم قیمت فعلی برابر با قیمت اولیه
        self.price_range = price_range  # ذخیره بازه قیمت
        self.max_steps = max_steps  # تعداد حداکثر گام‌ها در هر اپیزود
        self.step_count = 0  # شمارنده گام‌ها

    # متد برای اجرای عمل و برگرداندن وضعیت جدید، پاداش و شرط اتمام
    def step(self, action):
        self.step_count += 1  # افزایش شمارنده گام‌ها
        done = self.step_count >= self.max_steps  # تعیین شرط اتمام اپیزود
        self.current_price = random.uniform(*self.price_range)  # تولید قیمت جدید به صورت تصادفی
        reward = self._calculate_reward(action)  # محاسبه پاداش بر اساس عمل انجام شده
        return self._get_state_index(), reward, done  # برگرداندن شاخص وضعیت، پاداش و شرط اتمام

    # متد خصوصی برای محاسبه پاداش
    def _calculate_reward(self, action):
        if action == 1:  # اگر عمل خرید انجام شده
            reward = self.current_price - self.initial_price  # محاسبه سود
        elif action == 2:  # اگر عمل فروش انجام شده
            reward = self.initial_price - self.current_price  # محاسبه زیان
        else:  # اگر عمل نگه داشتن انجام شده
            reward = 0  # بدون پاداش یا زیان
        return reward  # برگرداندن پاداش

    # متد خصوصی برای نگاشت قیمت به شاخص مناسب در جدول Q
    def _get_state_index(self):
        state_index = int((self.current_price - self.price_range[0]) / (self.price_range[1] - self.price_range[0]) * (state_size - 1))
        return state_index  # برگرداندن شاخص وضعیت

    # متد برای ریست کردن محیط به وضعیت اولیه
    def reset(self):
        self.current_price = self.initial_price  # تنظیم مجدد قیمت فعلی به قیمت اولیه
        self.step_count = 0  # ریست شمارنده گام‌ها
        return self._get_state_index()  # برگرداندن شاخص وضعیت اولیه

# تعریف کلاس عامل دستیار معامله‌گر
class TradingAgent:
    # متد سازنده کلاس که با اندازه حالت و اقدامات فراخوانی می‌شود
    def __init__(self, state_size, action_size):
        self.state_size = state_size  # ذخیره اندازه حالت
        self.action_size = action_size  # ذخیره تعداد اقدامات ممکن
        self.q_table = np.zeros((state_size, action_size))  # ایجاد جدول Q با مقادیر اولیه صفر

    # متد برای انتخاب عمل بر اساس سیاست ε-greedy
    def choose_action(self, state_index):
        if random.uniform(0, 1) < 0.1:  # با احتمال 10% یک عمل تصادفی انتخاب می‌شود
            return random.choice([0, 1, 2])  # انتخاب تصادفی یکی از اقدامات
        else:  # با احتمال 90% بهترین عمل بر اساس جدول Q انتخاب می‌شود
            return np.argmax(self.q_table[state_index])  # انتخاب عمل با بیشترین ارزش

    # متد برای یادگیری و به‌روزرسانی جدول Q
    def learn(self, state_index, action, reward, next_state_index, done):
        if not done:  # اگر اپیزود به پایان نرسیده
            predict = self.q_table[state_index, action]  # پیش‌بینی فعلی
            target = reward + 0.9 * np.max(self.q_table[next_state_index])  # هدف بر اساس پاداش و ارزش بهترین عمل بعدی
            self.q_table[state_index, action] += 0.1 * (target - predict)  # به‌روزرسانی جدول Q

# تعریف اندازه حالت و اقدامات
state_size = 21  # اندازه حالت برای نمونه
action_size = 3  # تعداد اقدامات ممکن: نگه داشتن، خرید، فروش

# ایجاد محیط و عامل
env = TradingEnvironment()  # ایجاد نمونه از محیط
agent = TradingAgent(state_size, action_size)  # ایجاد نمونه از عامل

# حلقه اصلی برای آموزش عامل
for episode in range(10):  # تکرار برای 10 اپیزود
    state_index = env.reset()  # ریست محیط و دریافت شاخص وضعیت اولیه
    done = False  # تنظیم شرط اتمام به False

    while not done:  # تا زمانی که اپیزود به پایان نرسیده
        action = agent.choose_action(state_index)  # انتخاب عمل توسط عامل
        next_state_index, reward, done = env.step(action)  # اجرای عمل و دریافت وضعیت جدید، پاداش و شرط اتمام
        agent.learn(state_index, action, reward, next_state_index, done)  # یادگیری و به‌روزرسانی جدول Q
        state_index = next_state_index  # به‌روزرسانی وضعیت فعلی

    # چاپ جدول Q پس از هر اپیزود
    print(f"جدول Q پس از آموزش اپیزود {episode + 1}:")
    print(agent.q_table)

جدول Q پس از آموزش اپیزود 1:
[[ 1.12526486e-05  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.95009549e-02  0.00000000e+00  0.00000000e+00]
 [ 1.26860085e-01  0.00000000e+00  0.00000000e+00]
 [ 6.37993761e-04  0.00000000e+00  0.00000000e+00]
 [ 8.57539893e-02  0.00000000e+00  0.00000000e+00]
 [ 5.31532228e-02  0.00000000e+00  6.79596109e-01]
 [ 1.36217805e-01  0.00000000e+00  0.00000000e+00]
 [ 6.88116273e-05  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.08925012e-02  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.14861429e-02  0.00000000e+00  0.00000000e+00]
 [ 8.40097559e-03  0.00000000e+00  0.00000000e+00]
 [ 3.55508594e-04  0.00000000e+00  0.00000000e+00]
 [ 8.82457618e-04  0.00000000e+00  0.00000000e+00]
 [ 9.10445122e-05  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.83247516e-01  0.00000000e+00]
 [